### A jupyter notebook for analysing mixed sanger data files

#### Exploring the use of python to extract secondary sequences from mixed ab1 files

In [ ]:
from Bio import SeqIO
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import subprocess

In [ ]:
%matplotlib notebook

#### Set you abi folder path
<br>
Make sure you file are named appropreately, name-R and name-F

In [ ]:
ab1Folder = 'ab1/'

In [ ]:
ab1List = []
ab1PairDict = {}
keyList = []
traceSetDict = {}
for trace in os.listdir(ab1Folder):
    if ".ab1" in trace:
        trace = trace
        ab1List.append(trace)
        keyList.append(trace.replace("R", "").replace("F", "").replace(".ab1", ""))
        traceSetDict[trace] = trace.replace("R", "").replace("F", "").replace(".ab1", "")

keyList = list(set(keyList)) 

for key in keyList:
    pair = []
    for ab1 in ab1List:
        if key == ab1.replace("R", "").replace("F", "").replace(".ab1", ""):
            pair.append(ab1)
    ab1PairDict[key] = pair


print(ab1PairDict)

We now have a python dictionary with a key and a list of trace files.

### Run tracy to call bases

https://github.com/gear-genomics/tracy

tracy can also be used to decompose sequences, trying on a desktop PC did not work, requires more than 16 GB of ram for full chromosome 9.  It does work with just the gene of interest.

In [ ]:
for trace in ab1List:
    tracy_cmd = "./tracy basecall -f tsv -o output/{} ab1/{}".format(trace.replace(".ab1", ".tsv"), trace)
    print(tracy_cmd)
    #os.popen(tracy_cmd)
    subprocess.call(tracy_cmd, shell = True)
    
    

Import our results into a dataframe

In [ ]:
baseDf = pd.DataFrame()
for key in ab1PairDict:
    print(key)
    for ab1 in ab1PairDict[key]:
        print(ab1)
        
        tmpDf = pd.read_csv('output/{}.tsv'.format(ab1.replace(".ab1", "")), delimiter= "\t")
        tmpDf['key'] = key
        tmpDf['trace'] = ab1
        baseDf = baseDf.append(pd.DataFrame(data = tmpDf), ignore_index=True)

In [ ]:
baseDf.head()

Remove all positions where basenum is NA

In [ ]:
baseDf = baseDf[np.isfinite(baseDf['basenum'])]
baseDf = baseDf.loc[baseDf['qual'] > 9]

In [ ]:
baseDf.head()

Plot the quality values to get an idea

In [ ]:
fig=plt.figure(figsize=(12,8),dpi= 80, facecolor='w', edgecolor='k')

for trace in ab1List:

    plt.subplot(len(ab1List), 1, ab1List.index(trace)+1)
    plt.tight_layout(pad=0.4, h_pad=1)
    plotDf = baseDf.loc[baseDf['trace'] == trace]
    plt.scatter(x = plotDf['basenum'], y = plotDf['qual'], s = 2)
    plt.title(trace)
plt.suptitle('Quality of traces')
fig.subplots_adjust(top=0.88)
plt.show()

Explore the sanger file base on the plot above

### Extract the primary and secondary peaks

In [ ]:
primaryFasta = open("output/primary.fasta", "w")
for trace in ab1List:
    primaryDf = baseDf.loc[baseDf['trace'] == trace]
    primary = primaryDf['primary'].tolist()
    primary = ''.join(primary)
    print(">{}".format(trace))
    print(primary)
    primaryFasta.write(">{}".format(trace))
    primaryFasta.write('\n')
    primaryFasta.write(primary)
    primaryFasta.write('\n')
primaryFasta.close()

In [ ]:
secondaryFasta = open("output/secondary.fasta", "w")
for trace in ab1List:
    secondaryDf = baseDf.loc[baseDf['trace'] == trace]
    secondary = secondaryDf['secondary'].tolist()
    secondary = ''.join(secondary)
    print(">{}".format(trace))
    print(secondary)
    secondaryFasta.write(">{}".format(trace))
    secondaryFasta.write('\n')
    secondaryFasta.write(secondary)
    secondaryFasta.write('\n')
secondaryFasta.close()

### blast analysis

In [ ]:
!blastn -query output/primary.fasta -db ref/ABL -outfmt 6 -max_target_seqs 1 -out output/primary.blast
!blastn -query output/secondary.fasta -db ref/ABL -outfmt 6 -max_target_seqs 1 -out output/secondary.blast

#### Read in the blast results

In [ ]:
priDf = pd.read_csv('output/primary.blast', delimiter="\t", header=None,  usecols = [0,2,6,7,8,9])
priDf.head(n = -1)

In [ ]:
secDf = pd.read_csv('output/secondary.blast', delimiter="\t", header=None, usecols = [0, 2,6,7,8,9])
secDf.head(n = -1)

#### Plot the results

In [ ]:
fig=plt.figure(figsize=(12,2),dpi= 80, facecolor='w', edgecolor='k')
yPri = 1
ySec = 2
for index, row in priDf.iterrows():
    #print(row[6], row[7])
    plt.hlines(xmin=row[8], xmax=row[9], y=yPri, colors='blue')
    yPri = yPri +0.01
for index, row in secDf.iterrows():
    #print(row[6], row[7])
    plt.hlines(xmin=row[8], xmax=row[9], y=ySec, colors='red')
    ySec = ySec +0.01
plt.show()

The figure above shows the blast results for the primary (blue) and secondary (red).

In [ ]:
setList = []
for index, _ in priDf.iterrows():
    trace = priDf[0][index]
    key = traceSetDict[trace]
    setList.append(key)
priDf['key'] = setList

In [ ]:
setList = []
for index, _ in secDf.iterrows():
    trace = secDf[0][index]
    key = traceSetDict[trace]
    setList.append(key)
secDf['key'] = setList

In [ ]:
priDf.head(n = 100)

In [ ]:
secDf.head(n = 100)


In [ ]:
fig=plt.figure(figsize=(12,6),dpi= 80, facecolor='w', edgecolor='k')
yPri = 1
ySec = 2

for key in keyList:
    plt.subplot(len(keyList), 1, keyList.index(key)+1)
    plt.tight_layout(pad=1, h_pad=3)
    plotPriDf = priDf.loc[priDf['key'] == key]
    plotSecDf = secDf.loc[secDf['key'] == key]
    
    for index, row in plotPriDf.iterrows():
        #print(row[6], row[7])
        plt.hlines(xmin=row[8], xmax=row[9], y=yPri, colors='blue')
        yPri = yPri +0.04
    
    for index, row in plotSecDf.iterrows():
        #print(row[6], row[7])
        plt.hlines(xmin=row[8], xmax=row[9], y=ySec, colors='red')
        ySec = ySec +0.04
    plt.title(key)
    
plt.suptitle('Plot of BLAST results')
fig.subplots_adjust(top=0.88)
plt.show()

In [ ]:
!blastn -query output/primary.fasta -db ref/ABL -outfmt 2 


In [ ]:
!blastn -query output/secondary.fasta -db ref/ABL -outfmt 2

### Use tracy to decompose the ab1 files against the refernce

In [ ]:
!mkdir output/tracy

#### A python loop to handle tracy's decompose function for you

In [ ]:
for ab1 in ab1List:
    tracy_decompose_cmd = "./tracy decompose -g ref/ABL_ref.fasta -f align -o output/tracy/{} ab1/{}".format(ab1.replace(".ab1", ""), ab1) 
    print(tracy_decompose_cmd)
    #os.popen(tracy_decompose_cmd)
    subprocess.call(tracy_decompose_cmd, shell = True)

#### Your tracy decompose results will  be in output/tracy



In [ ]:
!ls output/tracy/